In [9]:
# Imports
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function
from zipfile import ZipFile
import os
from os.path import basename
import shutil
from shutil import make_archive
import datetime

#setting the random seed
rd.seed()

In [13]:
class Simple_EA_indstep_mutate:
    
    def __init__(self, budget):
        
        self.budget = budget
        self.d = 1
                
        # Parameter setting
        self.mu = 15 #number of parents
        self.lamda = 100 #number of offspring
        self.pc = 0.75 #recombination probability
        
    def initialize_individual(self):
        individual = np.empty((self.d,1))
        for i in range (0,self.d):
            individual[i]= rd.uniform(-5, 5)
        return individual

    def initialize_population(self):
        population = np.zeros((self.mu,self.d))
        for k in range(0,self.mu):
            population[k] = np.transpose(self.initialize_individual())
        return population

    def recombine(self,pop):
        offspring = np.zeros((self.d))
        p1 = pop[rd.randrange(0,len(pop))]
        p2 = pop[rd.randrange(0,len(pop))]
        if rd.uniform(0,1) < self.pc:
            for k in range (0,self.d):
                random_number = rd.uniform(0,1)
                if random_number < 0.5:
                    offspring[k] = p1[k]
                else:
                    offspring[k] = p2[k]
        else:
            offspring = p1
        return offspring
    
    def global_discrete_recombine(self, pop):
        offspring = np.zeros((self.d))
        for i in range(0, self.d):
            random_parent = pop[rd.randrange(0,len(pop))]
            offspring[i] = random_parent[i]
            
        return offspring

    def update_sigma(self, sigma, tau):
        new_sigma = sigma * math.exp(tau * np.random.normal(0,1))
        return new_sigma

    def singlestep_mutate (self,ind, sigma):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0, len(ind)):
            result = ind[j] + sigma * np.random.normal(0,1)
             # check boundaries first
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result

        return mutant
    
    def update_sigma_vec(self, sigma_vec, global_tau, local_tau):
        updated_sigma = np.zeros(self.d)
        for i in range (0,self.d):
            updated_sigma[i] = sigma_vec[i] * math.exp(global_tau * np.random.normal(0,1) + local_tau * np.random.normal(0,1))

        return updated_sigma

    def indstep_mutate(self, ind, sigma_vec):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0,len(ind)):
            result = ind[j] + sigma_vec[j] * np.random.normal(0,1)
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
                 # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result
 
        return mutant
    
    def __call__(self, func):

        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        
        #sigma = 10/6 #for singlestep mutation
        #tau = 1 / math.sqrt(self.d) #for singlestep mutation
        
        sigma = np.zeros(self.d) #for indstep mutation
        for s in range (0,self.d):
            sigma[s] = 10 / 6

        global_tau = 1 / math.sqrt(2*self.d); #for individual stepsize mutation
        local_tau = 1 / math.sqrt(2*math.sqrt(self.d)); #for individual stepsize mutation
        
        #Initialization
        xp = self.initialize_population() #parent population
        f = np.zeros((self.mu,1)) #parents fitness
        xo = np.zeros((self.lamda,self.d)) #offspring population
        fo = np.zeros((self.lamda,1)) #offspring fitness
        x_opt = None
        f_opt = np.inf
        
        for i in range (0,self.mu):
            f[i] = func(xp[i])

        # Evaluation loop
        while func.evaluations < eval_budget and func.final_target_hit == False :
            
            #update sigma
            sigma = self.update_sigma_vec(sigma, global_tau, local_tau)
            
            #create new offspring generation
            for i in range(0,self.lamda):
                offspring = self.global_discrete_recombine(xp)
                xo[i] = self.indstep_mutate(offspring, sigma)
                fo[i] = func(xo[i])
                
            # Selection
            m = np.concatenate((xo, fo), axis=1) # Create joint matrix of individuals and their fitness values
            sorted_m = m[np.argsort(m[:, self.d])] # sort based on fitness (lowest values highest ranking)
            
            xpnew = sorted_m[0:self.mu,0:self.d] # copy best individuals and remove fitness values
            xp = xpnew # set new population for next generation
            f = sorted_m[0:self.mu,self.d] # copy fitness values of new generation

            #check if new best individual is better than previous best and copy x and f if true
            if f[0] < f_opt:
                x_opt = xp[0]
                f_opt = f[0]

        return x_opt, f_opt

In [14]:
fIDs = []
for fid in range (1,25):
    fIDs.append(fid)

In [15]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1],[2,5,10], 1)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  "Simple_EA/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([Simple_EA_indstep_mutate(10000)])
#create a ZIP file for IOHprofiler upload
shutil.make_archive(path + experiment_name,"zip", path + experiment_name)

Running single-threaded
1 20015 2.676054078110596e-07
1 50015 0.1147443879336329
1 100015 10.845862543620052
2 20015 8.589059459900191e-06
2 50015 178.90839557181673
2 100015 7315.445812096598
3 20015 0.2405105352356788
3 50015 23.836852280465262
3 100015 43.820342682153886
4 20015 9.801790642211069
4 50015 32.7644567915648
4 100015 104.64852651576274
5 215 0.0
5 1715 0.0
5 54715 0.0
6 20015 0.0005028736082703481
6 50015 6.167357914993743
6 100015 86.71499475150081
7 20015 92.94000023956775
7 50015 103.68799871987696
7 100015 156.59638030904935
8 20015 0.02951669352893231
8 50015 87.11131950177288
8 100015 628.9697219757755
9 20015 0.03262507527717446
9 50015 60.63979799104179
9 100015 1148.3552252342495
10 20015 0.04774195858827866
10 50015 10038.175843541034
10 100015 22274.9354378533
11 20015 38.848534724685614
11 50015 31.939498196436173
11 100015 114.63057257716406
12 20015 0.43386619417953104
12 50015 240989.14382038836
12 100015 3299101.9980871873
13 20015 5.1707043033506634
13 

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/Simple_EA/experiment23-9-161932.zip'

In [1]:
from IOHexperimenter import IOH_function
f1 = IOH_function(1,2,1, suite='BBOB')

In [5]:
algorithm = Simple_EA_singlestep_mutate(4000)

In [6]:
algorithm(f1)

(array([ 0.25692116, -1.12692681]), 79.48090939162357)